In [12]:
import pmcx
import numpy as np
import mpmath as mp # mpmath documentation https://mpmath.org/doc/current/mpmath.pdf
import matplotlib.pyplot as plt

In [18]:
# Set desired precision (e.g., 128 decimal places)
mp.mp.dps = 128

def integral_klein_nishina(theta, wavelen):
    # Adapted from https://physics.stackexchange.com/questions/690255/angle-energy-dependent-cdf-for-compton-scattering

    cosTheta = mp.cos(theta)
    wavelen = mp.mpf(wavelen)

    h = mp.mpf(6.62607015e-34)
    c = mp.mpf(299792458)
    m = mp.mpf(9.10938356e-31)

    R = (h*wavelen)/(m*c**2)

    out = -(cosTheta/R**2) + mp.log(1 + R*(1-cosTheta)) * (1/R - 2/R**2 - 2/R**3)
    out = out - 1/(2*R*(1+R*(1-cosTheta))**2) + 1/(1+R*(1-cosTheta)) * (-2/R**2 - 1/R**3)

    return out

def cdf_klein_nishina(theta, wavelen):
    assert theta >= 0 and theta <= mp.pi
    out = (integral_klein_nishina(theta, wavelen) - integral_klein_nishina(0, wavelen))
    out = out / (integral_klein_nishina(mp.pi, wavelen) - integral_klein_nishina(0, wavelen))
    return out

def generate_theta_cdf_klein_nishina(wavelen):
    theta = np.linspace(0, np.pi, 100)
    cdf = [cdf_klein_nishina(t, wavelen) for t in theta]
    return (theta, cdf)

def plot_cdf_klein_nishina(wavelen):
    theta, cdf = generate_theta_cdf_klein_nishina(wavelen)
    print(cdf)
    plt.plot(theta, cdf)
    plt.show()

def testing_cdf_wavelengths():
    wavelengths = np.linspace(0.01, 1, 5)

    # Collect all the cdfs for each wavelength and then compare their absolute differences
    theta_cdfs = [generate_theta_cdf_klein_nishina(w) for w in wavelengths]
    for i in range(len(theta_cdfs)):
        for j in range(i+1, len(theta_cdfs)):
            theta1, cdf1 = theta_cdfs[i]
            theta2, cdf2 = theta_cdfs[j]
            diff = np.abs(np.array(cdf1) - np.array(cdf2))
            print(f"Max difference between {wavelengths[i]} and {wavelengths[j]} is {np.max(diff)}")

testing_cdf_wavelengths()

# TODO: because CDF is dependent on wavelength, and this changes after the first scattering, we need to update the CDF after each scattering event
# OR disable higher order scattering events



Max difference between 0.01 and 0.2575 is 0.0000000000000000000011265499859887544716604963110342923227248325800727692193398016603956402578053814628413029232786568925085401600207080219962134616
Max difference between 0.01 and 0.505 is 0.0000000000000000000022530999719775089038336755403959494310820339125740423028033309834496375053132613948768607777337523499962228014182017289174272353
Max difference between 0.01 and 0.7525 is 0.0000000000000000000033796499579662630833283332274265601762856322733257632576754256776410899456620531223783868110455839902943049877956801414390337126
Max difference between 0.01 and 1.0 is 0.0000000000000000000045061999439550177681576407878004753123651783276356598439653224152043920673893109048007372431304327165353590750627911893995744876
Max difference between 0.2575 and 0.505 is 0.0000000000000000000011265499859887544321731792293616571083572013325012730834635293230539972475078799320355578544550954574876826413974937069212137737
Max difference between 0.2575 and 0.75

In [ ]:
def run_simulation():
    res = pmcx.run(nphoton=1000000, 
                vol=np.ones([60, 60, 60], dtype='uint8'), 
                tstart=0, 
                tend=5e-9, 
                tstep=5e-9, 
                srcpos=[30,30,0], 
                srcdir=[0,0,1], 
                prop=np.array([[0, 0, 1, 1], [0.005, 1, 0.01, 1.37]]),
                )
    res['flux'].shape

    plt.imshow(np.log10(res['flux'][30,:, :]))
    plt.show()

run_simulation()